In [22]:
from pipeline import MongoDB_Connector
import os
from utils import flat_json_to_avro_schema, extract_schema_from_mongodb
import json
import fastavro

In [20]:
# Output folder
output_folder = "data_dict"

avro_schema_path = os.path.join(output_folder, "avro_schema.json")
if os.path.exists(avro_schema_path):
    avro_schema = json.load(open(avro_schema_path))
else:
    # MongoDB connection
    mongo = MongoDB_Connector()
    mongo.conn_mongo()
    # Extract and save schema
    extract_schema_from_mongodb(mongo.mongo_base_collection, output_folder, explore_limit = 20_000)

In [23]:
avro_schema = fastavro.parse_schema(avro_schema)

In [24]:
mongo = MongoDB_Connector()
mongo.conn_mongo()

2025-06-21 14:21 - INFO - Success in conn_mongo()


In [25]:
def normalize_record(record, schema):
    """Convert MongoDB record to Avro-ready format, dropping unwanted fields."""
    record = dict(record)
    record["_id"] = str(record.get("_id", ""))

    # Ensure all schema fields exist
    schema_fields = {f["name"] for f in schema["fields"]}
    for field in schema_fields:
        if field not in record:
            record[field] = None
    return record
batch_data = []
for doc in mongo.mongo_base_collection.find(limit = 20_000):
    # batch_data.append(normalize_record(doc, avro_schema))
    batch_data.append(normalize_record(doc, avro_schema))

In [27]:
batch_data[:5]

[{'_id': '5ed8cb2bc671fc36b74653ad',
  'time_stamp': 1591266092,
  'ip': '37.170.17.183',
  'user_agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 13_4_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1 Mobile/15E148 Safari/604.1',
  'resolution': '375x667',
  'user_id_db': '502567',
  'device_id': 'beb2cacb-20af-4f05-9c03-c98e54a1b71a',
  'api_version': '1.0',
  'store_id': '12',
  'local_time': '2020-06-04 12:21:27',
  'show_recommendation': 'false',
  'current_url': 'https://www.glamira.fr/glamira-pendant-viktor.html?alloy=yellow-375',
  'referrer_url': 'https://www.glamira.fr/men-s-necklaces/',
  'email_address': 'pereira.vivien@yahoo.fr',
  'recommendation': False,
  'utm_source': False,
  'utm_medium': False,
  'collection': 'view_product_detail',
  'product_id': '110474',
  'option': [{'option_label': 'alloy',
    'option_id': '332084',
    'value_label': '',
    'value_id': '3279318'},
   {'option_label': 'diamond',
    'option_id': '',
    'value_label': '',
   

In [28]:
import fastavro
with open('test.avro', "wb") as f:
    fastavro.writer(f, avro_schema, batch_data)